In [83]:
import pandas as pd

train_df = pd.read_csv("/kaggle/input/audios-afd/train_afd_audio.csv")


print(train_df.head())

                                                Ruta  Etiqueta
0  C:\Users\Usuario\AFD\MMUSED-fallacy\audio_clip...       0.0
1  C:\Users\Usuario\AFD\MMUSED-fallacy\audio_clip...       0.0
2  C:\Users\Usuario\AFD\MMUSED-fallacy\audio_clip...       0.0
3  C:\Users\Usuario\AFD\MMUSED-fallacy\audio_clip...       0.0
4  C:\Users\Usuario\AFD\MMUSED-fallacy\audio_clip...       0.0


In [84]:
train_df.iloc[0,:]

Ruta        C:\Users\Usuario\AFD\MMUSED-fallacy\audio_clip...
Etiqueta                                                  0.0
Name: 0, dtype: object

In [86]:
# Cambiar las rutas al formato de Kaggle

train_df['Ruta'] = train_df['Ruta'].str.replace(r'\\', '/', regex=True)
print(train_df, '\n')
train_df['Ruta'] = train_df['Ruta'].str.replace(
    'C:/Users/Usuario/AFD', 
    '/kaggle/input/datosss', 
    regex=True
)

print(train_df)


                                                    Ruta  Etiqueta
0      /kaggle/input/datosss/MMUSED-fallacy/audio_cli...       0.0
1      /kaggle/input/datosss/MMUSED-fallacy/audio_cli...       0.0
2      /kaggle/input/datosss/MMUSED-fallacy/audio_cli...       0.0
3      /kaggle/input/datosss/MMUSED-fallacy/audio_cli...       0.0
4      /kaggle/input/datosss/MMUSED-fallacy/audio_cli...       0.0
...                                                  ...       ...
17113  /kaggle/input/datosss/MMUSED-fallacy/audio_cli...       0.0
17114  /kaggle/input/datosss/MMUSED-fallacy/audio_cli...       0.0
17115  /kaggle/input/datosss/MMUSED-fallacy/audio_cli...       0.0
17116  /kaggle/input/datosss/MMUSED-fallacy/audio_cli...       1.0
17117  /kaggle/input/datosss/MMUSED-fallacy/audio_cli...       1.0

[17118 rows x 2 columns] 

                                                    Ruta  Etiqueta
0      /kaggle/input/datosss/MMUSED-fallacy/audio_cli...       0.0
1      /kaggle/input/datosss/MMUSE

In [87]:
train_df.Ruta

0        /kaggle/input/datosss/MMUSED-fallacy/audio_cli...
1        /kaggle/input/datosss/MMUSED-fallacy/audio_cli...
2        /kaggle/input/datosss/MMUSED-fallacy/audio_cli...
3        /kaggle/input/datosss/MMUSED-fallacy/audio_cli...
4        /kaggle/input/datosss/MMUSED-fallacy/audio_cli...
                               ...                        
17113    /kaggle/input/datosss/MMUSED-fallacy/audio_cli...
17114    /kaggle/input/datosss/MMUSED-fallacy/audio_cli...
17115    /kaggle/input/datosss/MMUSED-fallacy/audio_cli...
17116    /kaggle/input/datosss/MMUSED-fallacy/audio_cli...
17117    /kaggle/input/datosss/MMUSED-fallacy/audio_cli...
Name: Ruta, Length: 17118, dtype: object

In [88]:
import librosa
import numpy as np
import cv2

def audio_to_mel_spectrogram(path, sr=22050, n_mels=128, fmax=8000, duration=3):
    y, _ = librosa.load(path, sr=sr, duration=duration)
    
    # Padding si el audio es corto
    if len(y) < sr * duration:
        y = np.pad(y, (0, sr * duration - len(y)))
        
    mel_spec = librosa.feature.melspectrogram(y=y, sr=sr, n_mels=n_mels, fmax=fmax)
    mel_spec_db = librosa.power_to_db(mel_spec, ref=np.max)
    
    # Normalizar entre 0 y 255 para usarlo como imagen
    mel_spec_db = (mel_spec_db - mel_spec_db.min()) / (mel_spec_db.max() - mel_spec_db.min()) * 255
    mel_spec_db = mel_spec_db.astype(np.uint8)
    
    # Redimensionar para tener un tamaño fijo (por ejemplo 128x128)
    mel_spec_db = cv2.resize(mel_spec_db, (128, 128))
    
    # Convertir a 3 canales si querés una "imagen RGB"
    mel_spec_db = np.stack([mel_spec_db]*3, axis=-1)
    
    return mel_spec_db

In [90]:
X = []
y = []

for _, row in train_df.iterrows():
    spec = audio_to_mel_spectrogram(row['Ruta'])
    X.append(spec)
    y.append(row['Etiqueta'])

X = np.array(X)
y = np.array(y)


In [91]:
from sklearn.model_selection import train_test_split

# Divide el dataset en 80% train y 20% validación
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, stratify=y, random_state=42)

In [105]:

# Definición de la métrica F1 personalizada
class F1Metric(tf.keras.metrics.Metric):
    def __init__(self, name='f1_score', **kwargs):
        super(F1Metric, self).__init__(name=name, **kwargs)
        # No necesitamos 'shape' como argumento adicional.
        self.true_positives = self.add_weight(name='tp', initializer='zeros')
        self.false_positives = self.add_weight(name='fp', initializer='zeros')
        self.false_negatives = self.add_weight(name='fn', initializer='zeros')

    def update_state(self, y_true, y_pred, sample_weight=None):
        # Redondear las predicciones para obtener 0 o 1
        y_pred = tf.round(y_pred)

        # Calcular los verdaderos positivos, falsos positivos y falsos negativos
        self.true_positives.assign_add(tf.reduce_sum(y_true * y_pred))
        self.false_positives.assign_add(tf.reduce_sum((1 - y_true) * y_pred))
        self.false_negatives.assign_add(tf.reduce_sum(y_true * (1 - y_pred)))

    def result(self):
        # Calcular el F1 score
        precision = self.true_positives / (self.true_positives + self.false_positives + tf.keras.backend.epsilon())
        recall = self.true_positives / (self.true_positives + self.false_negatives + tf.keras.backend.epsilon())
        return 2 * (precision * recall) / (precision + recall + tf.keras.backend.epsilon())

    def reset_states(self):
        # Restablecer las métricas al final de cada época
        self.true_positives.assign(0)
        self.false_positives.assign(0)
        self.false_negatives.assign(0)


In [106]:
import tensorflow as tf
from tensorflow.keras import layers, models, metrics

# Definir el modelo CNN
def build_cnn(input_shape=(128, 128, 3)):
    model = models.Sequential([
        layers.Conv2D(32, (3, 3), activation='relu', input_shape=input_shape),
        layers.MaxPooling2D((2, 2)),
        layers.Conv2D(64, (3, 3), activation='relu'),
        layers.MaxPooling2D((2, 2)),
        layers.Conv2D(128, (3, 3), activation='relu'),
        layers.MaxPooling2D((2, 2)),
        layers.Flatten(),
        layers.Dense(128, activation='relu'),
        layers.Dropout(0.5),
        layers.Dense(1, activation='sigmoid')  # binaria
    ])
    model.compile(
        optimizer='adam',
        loss='binary_crossentropy',
        metrics=[metrics.AUC(), F1Metric(), 'accuracy']
    )
    return model


In [122]:

# Crear el modelo
model = build_cnn()
model.summary()

from sklearn.utils import class_weight
import numpy as np

# Calcular los pesos de las clases
class_weights = class_weight.compute_class_weight(
    class_weight='balanced',
    classes=np.unique(y_train),
    y=y_train
)

# Crear un diccionario de pesos de clase
class_weight_dict = {i: class_weights[i] for i in range(len(class_weights))}

# Ahora puedes pasar este diccionario de pesos al modelo durante el entrenamiento
model.fit(X_train, y_train, epochs=20, batch_size=32, validation_data=(X_val, y_val), class_weight=class_weight_dict)



Model: "sequential_9"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ conv2d_27 (Conv2D)                   │ (None, 126, 126, 32)        │             896 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling2d_27 (MaxPooling2D)      │ (None, 63, 63, 32)          │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d_28 (Conv2D)                   │ (None, 61, 61, 64)          │          18,496 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling2d_28 (MaxPooling2D)      │ (None, 30, 30, 64)          │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d_29 (Conv2D)                   │ (None, 28, 28, 128)         │          73,856 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling2d_29 (MaxPooling2D)      │ (None, 14, 14, 128)         │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ flatten_9 (Flatten)                  │ (None, 25088)               │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_18 (Dense)                     │ (None, 128)                 │       3,211,392 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_9 (Dropout)                  │ (None, 128)                 │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_19 (Dense)                     │ (None, 1)                   │             129 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 3,304,769 (12.61 MB)

 Trainable params: 3,304,769 (12.61 MB)

 Non-trainable params: 0 (0.00 B)

Epoch 1/20
428/428 ━━━━━━━━━━━━━━━━━━━━ 11s 16ms/step - accuracy: 0.4457 - auc_8: 0.5334 - f1_score: 0.1585 - loss: 3.1973 - val_accuracy: 0.3493 - val_auc_8: 0.5404 - val_f1_score: 0.1626 - val_loss: 0.6977
Epoch 2/20
428/428 ━━━━━━━━━━━━━━━━━━━━ 4s 10ms/step - accuracy: 0.3551 - auc_8: 0.5438 - f1_score: 0.1584 - loss: 0.6956 - val_accuracy: 0.0993 - val_auc_8: 0.5435 - val_f1_score: 0.1679 - val_loss: 0.7246
Epoch 3/20
428/428 ━━━━━━━━━━━━━━━━━━━━ 4s 10ms/step - accuracy: 0.1611 - auc_8: 0.5063 - f1_score: 0.1684 - loss: 0.7530 - val_accuracy: 0.3575 - val_auc_8: 0.5512 - val_f1_score: 0.1618 - val_loss: 0.6890
Epoch 4/20
428/428 ━━━━━━━━━━━━━━━━━━━━ 4s 10ms/step - accuracy: 0.3255 - auc_8: 0.5490 - f1_score: 0.1645 - loss: 0.6925 - val_accuracy: 0.3855 - val_auc_8: 0.5426 - val_f1_score: 0.1623 - val_loss: 0.6779
Epoch 5/20
428/428 ━━━━━━━━━━━━━━━━━━━━ 4s 10ms/step - accuracy: 0.3529 - auc_8: 0.5337 - f1_score: 0.1618 - loss: 0.7246 - val_accuracy: 0.3513 - val_auc_8: 0.5448 - val_

In [123]:
test_df = pd.read_csv("/kaggle/input/audios-afd/test_afd_audio.csv")

# Cambiar las rutas a formato de Kaggle (si no lo has hecho antes)
test_df['Ruta'] = test_df['Ruta'].str.replace(r'\\', '/', regex=True)
test_df['Ruta'] = test_df['Ruta'].str.replace('C:/Users/Usuario/AFD', '/kaggle/input/datosss', regex=True)

# Preparar los datos de test
X_test = []
for _, row in test_df.iterrows():
    spec = audio_to_mel_spectrogram(row['Ruta'])
    X_test.append(spec)

X_test = np.array(X_test)


/tmp/ipykernel_31/3900068870.py:16: RuntimeWarning: invalid value encountered in divide
  mel_spec_db = (mel_spec_db - mel_spec_db.min()) / (mel_spec_db.max() - mel_spec_db.min()) * 255
/tmp/ipykernel_31/3900068870.py:17: RuntimeWarning: invalid value encountered in cast
  mel_spec_db = mel_spec_db.astype(np.uint8)


In [124]:
X_test[1]

array([[[ 99,  99,  99],
        [ 91,  91,  91],
        [ 97,  97,  97],
        ...,
        [  0,   0,   0],
        [  0,   0,   0],
        [  0,   0,   0]],

       [[109, 109, 109],
        [ 95,  95,  95],
        [100, 100, 100],
        ...,
        [  0,   0,   0],
        [  0,   0,   0],
        [  0,   0,   0]],

       [[121, 121, 121],
        [103, 103, 103],
        [103, 103, 103],
        ...,
        [  0,   0,   0],
        [  0,   0,   0],
        [  0,   0,   0]],

       ...,

       [[ 40,  40,  40],
        [ 55,  55,  55],
        [ 88,  88,  88],
        ...,
        [  0,   0,   0],
        [  0,   0,   0],
        [  0,   0,   0]],

       [[ 39,  39,  39],
        [ 49,  49,  49],
        [ 75,  75,  75],
        ...,
        [  0,   0,   0],
        [  0,   0,   0],
        [  0,   0,   0]],

       [[ 32,  32,  32],
        [ 23,  23,  23],
        [ 37,  37,  37],
        ...,
        [  0,   0,   0],
        [  0,   0,   0],
        [  0,   0,   0]]

In [125]:
# Realizar las predicciones
predicciones = model.predict(X_test)


68/68 ━━━━━━━━━━━━━━━━━━━━ 1s 6ms/step


In [126]:
# Convertir probabilidades a clases (0 o 1) usando un umbral de 0.5
y_pred = (predicciones > 0.3).astype(int)

In [127]:
y_pred

array([[1],
       [1],
       [1],
       ...,
       [0],
       [0],
       [0]])

In [128]:
import numpy as np

# Contar las ocurrencias de cada clase (0 y 1) en y_pred
conteo = np.bincount(y_pred.flatten())  # Aplanar para asegurar que es un vector 1D
print(f"Etiquetas 0: {conteo[0]}")
print(f"Etiquetas 1: {conteo[1]}")

Etiquetas 0: 497
Etiquetas 1: 1678


In [129]:
import pandas as pd

# Supongamos que y_pred es un array numpy
# Si querés que la columna tenga un nombre específico, podés pasarlo como un diccionario o una lista
df = pd.DataFrame(y_pred, columns=["Predicción"])

# Guardar en CSV
df.to_csv("/kaggle/working/predicciones.csv", index=False)